In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
#Hasta 15 de Abril (para entrenar)

postulantes_edu_df = pd.read_csv('fiuba_1_postulantes_educacion_h15.csv')
postulantes_gye_df = pd.read_csv('fiuba_2_postulantes_genero_y_edad_h15.csv')
vistas_df = pd.read_csv('fiuba_3_vistas_h15.csv')
postulaciones_df = pd.read_csv('fiuba_4_postulaciones_h15.csv')
avisos_online_df = pd.read_csv('fiuba_5_avisos_online_h15.csv')
avisos_detalle_df = pd.read_csv('fiuba_6_avisos_detalle_h15.csv')

In [3]:
#Originales TP1 (para entrenar)

postulantes_edu_df2 = pd.read_csv('fiuba_1_postulantes_educacion.csv')
postulantes_gye_df2 = pd.read_csv('fiuba_2_postulantes_genero_y_edad.csv')
vistas_df2 = pd.read_csv('fiuba_3_vistas.csv')
postulaciones_df2 = pd.read_csv('fiuba_4_postulaciones.csv')
avisos_online_df2 = pd.read_csv('fiuba_5_avisos_online.csv')
avisos_detalle_df2 = pd.read_csv('fiuba_6_avisos_detalle.csv')

In [4]:
#Realizo el Merge del tipo outer de los datos
#Originales TP1 y Datos hasta 15 de abril

postulantes_edu_df = postulantes_edu_df.merge(postulantes_edu_df2,how='outer')
postulantes_gye_df = postulantes_gye_df.merge(postulantes_gye_df2,how='outer')
vistas_df = vistas_df.merge(vistas_df2,how='outer')
postulaciones_df = postulaciones_df.merge(postulaciones_df2,how='outer')
avisos_online_df = avisos_online_df.merge(avisos_online_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_df2,how='outer')

In [5]:
#Desde 15 de abril (para entrenar)

postulantes_edu_df2 = pd.read_csv('fiuba_1_postulantes_educacion_d15.csv')
postulantes_gye_df2 = pd.read_csv('fiuba_2_postulantes_genero_y_edad_d15.csv')
vistas_df2 = pd.read_csv('fiuba_3_vistas_d15.csv')
avisos_detalle_df2 = pd.read_csv('fiuba_6_avisos_detalle_d15.csv')

In [6]:
#Avisos detalle que faltaban
avisos_detalle_missing = pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

In [7]:
#Mergeamos todo

postulantes_edu_df = postulantes_edu_df.merge(postulantes_edu_df2,how='outer')
postulantes_gye_df = postulantes_gye_df.merge(postulantes_gye_df2,how='outer')
vistas_df = vistas_df.merge(vistas_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_missing,how='outer')

# Libero Espacio de la Memoria

In [8]:
del postulantes_edu_df2
del postulantes_gye_df2
del vistas_df2
del avisos_detalle_df2

# Trabajamos con la educacion

Asignamos un valor para cada estado de la educacion, Graduado->2, En Curso->1, Abandonado->0

In [9]:
def setCategory(c):
    if c == 'Graduado':
        return 2
    elif c == 'En Curso':
        return 1
    return 0 

In [10]:
postulantes_edu_df['valor_estado'] = postulantes_edu_df['estado'].apply(setCategory)

In [11]:
postulantes_edu_df.head(2)

,idpostulante,nombre,estado,valor_estado
0,ZjlZ,Master,En Curso,1
1,NdJl,Posgrado,En Curso,1


Al igual que hicimos con los estados de la educacion, lo hacemos con el nivel de educacion.

In [12]:
postulantes_edu_df['nombre'].value_counts()

Universitario        250850
Secundario           244689
Terciario/Técnico    113421
Otro                  53363
Posgrado              20624
Master                10074
Doctorado               620
Name: nombre, dtype: int64

In [13]:
def setCategory(c):
    
    if c == 'Doctorado':
        return 12
    elif c == 'Master':
        return 10
    elif c == 'Posgrado':
        return 8
    elif c == 'Universitario':
        return 6
    elif c == 'Terciario/Técnico':
        return 4
    elif c == 'Secundario':
        return 2
    return 0 

In [14]:
postulantes_edu_df['valor_nombre'] = postulantes_edu_df['nombre'].apply(setCategory)

In [15]:
postulantes_edu_df.head(2)

,idpostulante,nombre,estado,valor_estado,valor_nombre
0,ZjlZ,Master,En Curso,1,10
1,NdJl,Posgrado,En Curso,1,8


Ahora unificamos los datos obtenidos, creando una escala donde a mayor educacion, mayor peso. Considerando el tipo de educacion y su estado.
Este nuevo feature llamado valoracion_edu, reemplazará a 'nombre'y 'estado'

In [16]:
postulantes_edu_df['valoracion_edu'] = postulantes_edu_df['valor_estado'] + postulantes_edu_df['valor_nombre']
postulantes_edu_df.drop(columns=['valor_estado','valor_nombre','nombre','estado'],axis=1,inplace=True)
postulantes_edu_df.sort_values(by='valoracion_edu',ascending=False,inplace=True)
postulantes_edu_df.drop_duplicates(inplace=True)
postulantes_edu_df.head()

,idpostulante,valoracion_edu
318724,N9AN45,14
128733,vV9Ewjj,14
337840,aMbWjD,14
398665,OqmK81N,14
14753,JBrVvBp,14


Para la informacion sobre el genero, unificamos los valores 0.0 con los NO_DECLARA

In [17]:
postulantes_gye_df.sexo.value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
0.0                7
Name: sexo, dtype: int64

In [18]:
def setCategory(c):
    if c == '0.0':
        return 'NO_DECLARA'
    return c

In [19]:
postulantes_gye_df['sexo'] = postulantes_gye_df['sexo'].apply(setCategory)

In [20]:
postulantes_gye_df.sexo.value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25943
Name: sexo, dtype: int64

# Unificamos los dataframes GYE con EDU

In [21]:
postulantes_filtrados_df = postulantes_gye_df.merge(postulantes_edu_df,on='idpostulante',how='outer')

print('Shape',postulantes_filtrados_df.shape)

print('Unique Shape',postulantes_filtrados_df['idpostulante'].unique().shape)

Shape (738053, 4)
Unique Shape (504407,)


In [22]:
postulantes_filtrados_df.head(2)

,idpostulante,fechanacimiento,sexo,valoracion_edu
0,6MM,1985-01-01,MASC,NaN
1,Nzz,NaN,NO_DECLARA,NaN


Asumimos que los usuarios que no completaron su valor, tienen educacion minima, es decir, secundario abandonado.
Las fechas de nacimiento nulas se completa con la edad promedio de los usuarios registrados(31 años).

In [23]:
postulantes_filtrados_df['sexo'].fillna('NO_DECLARA',inplace=True)
postulantes_filtrados_df['valoracion_edu'].fillna(2, inplace=True)
postulantes_filtrados_df['fechanacimiento'].fillna('1987-01-01', inplace=True)

In [24]:
postulantes_filtrados_df.head(2)

,idpostulante,fechanacimiento,sexo,valoracion_edu
0,6MM,1985-01-01,MASC,2.0
1,Nzz,1987-01-01,NO_DECLARA,2.0


Al ordenar alfabeticamente sexo, los No_declara quedan ultimos, por lo tanto, al hacer un drop_duplicates, nos quedamos con la primera aparicion de ese usuario.
De esta forma si un usuario se registro dos veces, queda el registro correcto sobre su genero.

In [25]:
postulantes_filtrados_df.sort_values(by='sexo',inplace=True)

In [26]:
postulantes_filtrados_df = postulantes_filtrados_df.drop_duplicates('idpostulante')
postulantes_filtrados_df.shape

(504407, 4)

In [27]:
postulantes_filtrados_df.sexo.value_counts()

FEM           251226
MASC          227832
NO_DECLARA     25349
Name: sexo, dtype: int64

In [28]:
postulantes_filtrados_df.idpostulante.unique().shape

(504407,)

Las edades incorrectas son reemplazadas por al edad promedio.

In [29]:
postulantes_edad_trucha = postulantes_filtrados_df[ (postulantes_filtrados_df['fechanacimiento'].str.startswith('0')) == True]
postulantes_edad_trucha['fechanacimiento'] = '1987-01-01'
postulantes_edad_trucha.head()


C:\Users\Andres\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,idpostulante,fechanacimiento,sexo,valoracion_edu
299748,8M2R6pz,1987-01-01,FEM,2.0
311804,0zkAXBE,1987-01-01,FEM,7.0
341475,VNrP8EE,1987-01-01,FEM,2.0
347580,xkdB9JK,1987-01-01,FEM,8.0
324043,VNxMRxj,1987-01-01,FEM,2.0


Filtro la edad incorrecta y hago un append de la edad corregida.

In [30]:
postulantes_filtrados_df = postulantes_filtrados_df[ (postulantes_filtrados_df['fechanacimiento'].str.startswith('0')) == False]
postulantes_filtrados_df = postulantes_filtrados_df.append(postulantes_edad_trucha)
postulantes_filtrados_df.head(2)


,idpostulante,fechanacimiento,sexo,valoracion_edu
369026,5mPYWjp,1989-05-01,FEM,4.0
588433,xkdxkDz,1993-08-28,FEM,5.0


Verifico que no se hayan perdido datos.

In [31]:
postulantes_filtrados_df.shape

(504407, 4)

Calculo la edad y reemplazo la columna 'fechanacimiento' por 'edad'

In [32]:
postulantes_filtrados_df.loc[:,'fechanacimiento'] = pd.to_datetime(postulantes_filtrados_df['fechanacimiento']) 
postulantes_filtrados_df.loc[:,'edad'] = (2018 - postulantes_filtrados_df.fechanacimiento.dt.year)
postulantes_filtrados_df.drop(columns='fechanacimiento',axis=1,inplace=True)
postulantes_filtrados_df.head(2)

,idpostulante,sexo,valoracion_edu,edad
369026,5mPYWjp,FEM,4.0,29
588433,xkdxkDz,FEM,5.0,25


# Para exportar e importar el set de postulantes filtrados

postulantes_filtrados_df.to_csv('postulantes_filtrados_df.csv')

postulantes_filtrados_df = pd.read_csv('postulantes_filtrados_df.csv')
postulantes_filtrados_dfdrop('Unnamed: 0',1,inplace=True)

# Avisos Detalle

In [34]:
avisos_detalle_df.head(2)

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,1,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,1,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan


In [35]:
print(avisos_detalle_df.shape)
print(avisos_detalle_df['idaviso'].unique().shape)

(25901, 11)
(25288,)


Sacamos los repetidos de avisos_detalle

In [36]:
avisos_detalle_filtrados_df = avisos_detalle_df.drop_duplicates('idaviso')

In [37]:
del avisos_detalle_df

Hago un drop de las columnas que no voy a utilizar.

In [38]:
avisos_detalle_filtrados_df = avisos_detalle_filtrados_df.drop(columns=['idpais','mapacalle','ciudad','descripcion'],axis=1)

In [39]:
avisos_detalle_filtrados_df.head(2)

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Responsable de Marketing-Producto Div. Constru...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Asesor de Servicios - Postventa Chevrolet,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan


Unificamos zonas cercanas

In [40]:
avisos_detalle_filtrados_df.nombre_zona.value_counts()

Gran Buenos Aires              23018
Capital Federal                 2182
Buenos Aires (fuera de GBA)       57
GBA Oeste                         10
La Plata                           4
Ciudad de Mendoza                  3
Mendoza                            3
Rosario                            2
Cordoba                            2
Catamarca                          1
Neuquen                            1
San Juan                           1
Santa Cruz                         1
Tucuman                            1
Corrientes                         1
Santa Fe                           1
Name: nombre_zona, dtype: int64

In [41]:
def agrupacionZona(c):
    
    if c == 'Capital Federal' or c=='GBA Oeste':
        return 'Gran Buenos Aires'
    
    elif c == 'Ciudad de Mendoza':
        return 'Mendoza'
    
    return c 

In [42]:
avisos_detalle_filtrados_df['nombre_zona'] = avisos_detalle_filtrados_df['nombre_zona'].apply(agrupacionZona)

In [43]:
avisos_detalle_filtrados_df.nombre_zona.value_counts()

Gran Buenos Aires              25210
Buenos Aires (fuera de GBA)       57
Mendoza                            6
La Plata                           4
Rosario                            2
Cordoba                            2
Catamarca                          1
Neuquen                            1
San Juan                           1
Santa Cruz                         1
Tucuman                            1
Corrientes                         1
Santa Fe                           1
Name: nombre_zona, dtype: int64

In [44]:
avisos_detalle_filtrados_df['nivel_laboral'] = pd.Categorical(avisos_detalle_filtrados_df['nivel_laboral'])
avisos_detalle_filtrados_df['nivel_laboral'] = avisos_detalle_filtrados_df['nivel_laboral'].cat.codes
avisos_detalle_filtrados_df['nivel_laboral'].value_counts()

 4    16980
 2     4149
 3     1975
 1     1527
-1      335
 0      322
Name: nivel_laboral, dtype: int64

In [45]:
avisos_detalle_filtrados_df['tipo_de_trabajo'] = pd.Categorical(avisos_detalle_filtrados_df['tipo_de_trabajo'])
avisos_detalle_filtrados_df['tipo_de_trabajo'] = avisos_detalle_filtrados_df['tipo_de_trabajo'].cat.codes
avisos_detalle_filtrados_df['tipo_de_trabajo'].value_counts()

1    22831
2     1746
7      248
5      125
3      119
8       96
4       88
0       28
6        6
9        1
Name: tipo_de_trabajo, dtype: int64

# Para exportar e importar Avisos Detalle Filtrados

avisos_detalle_filtrados_df.to_csv('avisos_detalle_filtrados_df.csv')

avisos_detalle_filtrados_df = pd.read_csv('avisos_detalle_filtrados_df.csv')

avisos_detalle_filtrados_df.drop('Unnamed: 0',1,inplace=True)

# Postulaciones

Nos quedamos con los usuarios sobre los cuales tenemos informacion y ademas se postularon a un aviso.

In [47]:
postulaciones_df.head(2)

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39


Comparamos los "usuarios" con los  postulantes.

In [48]:
print(postulantes_filtrados_df.idpostulante.unique().shape)
print(postulaciones_df.idpostulante.unique().shape)

(504407,)
(348883,)


Como resultado del siguiente merge tenemos los postulantes que coinciden en postulaciones y postulantes.
Observamos que hay usuarios que no se postularon a ningun aviso.

In [49]:
postulantes_filtrados_efectivos_df = postulaciones_df.merge(postulantes_filtrados_df,on='idpostulante',how='inner')
postulantes_filtrados_efectivos_df.idpostulante.unique().shape

(348883,)

In [50]:
postulantes_filtrados_efectivos_df =  postulantes_filtrados_efectivos_df.drop(['idaviso','fechapostulacion'],axis='columns')
postulantes_filtrados_efectivos_df = postulantes_filtrados_efectivos_df.reset_index()
postulantes_filtrados_efectivos_df =  postulantes_filtrados_efectivos_df.drop('index',axis='columns')
postulantes_filtrados_efectivos_df.head(2)

,idpostulante,sexo,valoracion_edu,edad
0,NjlD,FEM,6.0,37
1,ZaO5,FEM,6.0,40


In [51]:
postulantes_filtrados_efectivos_df.shape

(6604534, 4)

# Para exportar e importar postulantes_filtrados_efectivos

postulantes_filtrados_efectivos_df.to_csv('postulantes_filtrados_efectivos_df.csv')

postulantes_filtrados_efectivos_df = pd.read_csv('postulantes_filtrados_efectivos_df.csv')
postulantes_filtrados_efectivos_df.drop('Unnamed: 0',1,inplace=True)

# Armado del set de Postulaciones

Me quedo con los idavisos sobre los cuales tengo informacion

In [53]:
avisos = avisos_detalle_filtrados_df['idaviso'].to_frame()

Me quedo con los idpostulantes sobre los cuales tengo informacion

In [54]:
postulantes = postulantes_filtrados_df['idpostulante'].to_frame()

Hago una interseccion para quedarme con usuarios y avisos sobre los cuales tengo informacion de su postulacion

In [55]:
postulantes_df =  postulaciones_df.merge(avisos,how='inner',on='idaviso')
postulantes_df = postulantes_df.merge(postulantes,how='inner',on='idpostulante')

Armo el formato necesario para que me quede la dupla idpostulante-idaviso

In [56]:
postulantes_df =postulantes_df.groupby('idpostulante').agg({'idaviso':'value_counts'})
postulantes_df.columns=['postulacion']
postulantes_df['postulacion'] = 1 

Borramos para liberar memoria informacion que ya no utilizamos

In [57]:
postulantes_df.reset_index(inplace=True)

In [58]:
postulantes_df.head(2)

,idpostulante,idaviso,postulacion
0,0z5Dmrd,1112384041,1
1,0z5Dmrd,1112420060,1


In [59]:
postulantes_df.idaviso.unique().shape

(18695,)

Ahora vamos a limitar la cantidad de postulaciones por usuario, para evitar aprender demasiado sobre pocos usuarios con muchas postulaciones. Elegimos limitarlas al promedio de postulaciones por usuario (20).

Agrupamos por idpostulante para poder acceder a cada usuario en particular de forma directa

In [60]:
grupo_postulantes = postulantes_df.groupby('idpostulante')

Ejemplo de cómo accedemos al usuario y aprovechamos el resultado para generar el DataFrame base.

In [61]:
postulantes_unicos = postulantes_df.idpostulante.unique()

In [62]:
postulantes_df = grupo_postulantes.get_group('0z5Dmrd').head(20).drop([0,1],axis=0)

In [76]:
for postulante in postulantes_unicos:
     postulantes_df = postulantes_df.append(grupo_postulantes.get_group(postulante).head(20))      

In [63]:
postulantes_df.head()

,idpostulante,idaviso,postulacion


# Para exportar e importar postulantes limitados

In [ ]:
postulantes_df.to_csv('postulaciones_limitadas.csv')

In [64]:
postulantes_df = pd.read_csv('postulaciones_limitadas.csv')
postulantes_df = postulantes_df.drop('Unnamed: 0',1)

In [65]:
postulantes_df.head(2)

,idpostulante,idaviso,postulacion
0,0z5Dmrd,1112384041,1
1,0z5Dmrd,1112420060,1


# Armado del set de No Postulaciones

Armarmos un set de no postulaciones asignando de forma random avisos a usuarios.
Mas adelante se verifica que esta no postulacion al azar no sea una postulacion efectiva o que el aviso no haya sido visto por el usuario.

In [66]:
vistas_df.head(2)

,idAviso,timestamp,idpostulante
0,1112400775,2018-04-06T01:43:47.445-0400,kPjr2K0
1,1112400775,2018-04-06T01:43:48.330-0400,kPjr2K0


In [67]:
vistas_df = vistas_df.drop('timestamp',axis=1)
vistas_df['lo_vio']=1
vistas_df.head(2)

,idAviso,idpostulante,lo_vio
0,1112400775,kPjr2K0,1
1,1112400775,kPjr2K0,1


In [68]:
vistas_df.columns = ['idaviso','idpostulante','lo_vio']


In [69]:
vistas_df = vistas_df.drop_duplicates(['idaviso','idpostulante'])

In [70]:
vistas_df.shape

(8597915, 3)

Asumimos que si un usuario no aparece en el DataFrame de vistas o en postulaciones, es un usuario inactivo. Por eso tomamos los usuarios que aparecen en alguno de los DataFrames mencionados

In [88]:
lista = []
usuarios_activos = vistas_df.drop_duplicates('idpostulante').merge(postulaciones_df.drop_duplicates('idpostulante').drop(['idaviso','fechapostulacion'],1),how='outer',on='idpostulante')
user_list = postulantes_filtrados_df.merge(usuarios_activos,how='inner',on='idpostulante').idpostulante.unique()
aviso_list = avisos_detalle_filtrados_df['idaviso'].unique()
index = 1
for n in range(3000000):
    user_index = random.randint(0,476360)
    aviso_index = random.randint(0,25287)
    lista.append((index,user_list[user_index],aviso_list[aviso_index],0))
    index+=1        

In [89]:
no_postulantes_df = pd.DataFrame(lista)

In [90]:
del lista

In [92]:
no_postulantes_df.head(2)

,0,1,2,3
0,1,NzrLwKJ,1112513555,0
1,2,N6qMwV,1111882745,0
2,3,Nzr3zvL,1112362954,0
3,4,jkj13j0,1112146217,0
4,5,ZDrmWxY,1112400202,0


In [93]:
no_postulantes_df.drop(0,axis='columns',inplace=True)

In [95]:
no_postulantes_df.columns= ['idpostulante','idaviso','postulacion']

In [94]:
no_postulantes_df.head()

,1,2,3
0,NzrLwKJ,1112513555,0
1,N6qMwV,1111882745,0
2,Nzr3zvL,1112362954,0
3,jkj13j0,1112146217,0
4,ZDrmWxY,1112400202,0


In [96]:
no_postulantes_df.shape

(3000000, 3)

# Armado de Set de Entrenamiento

Quitamos las no postulaciones que figuran como vistas.

In [99]:
entrenamiento_df = vistas_df.merge(no_postulantes_df,how='outer',on=['idpostulante','idaviso'])

In [100]:
entrenamiento_df.head()

,idaviso,idpostulante,lo_vio,postulacion
0,1112400775,kPjr2K0,1.0,NaN
1,1112426664,GNOwZmb,1.0,NaN
2,1112445738,lDLxajz,1.0,NaN
3,1112416123,owzZOqe,1.0,NaN
4,1112443938,NGk68l,1.0,NaN


In [101]:
entrenamiento_df['lo_vio'].hasnans

True

In [102]:
entrenamiento_df['postulacion'].fillna(0,inplace=True)
entrenamiento_df['lo_vio'].fillna(0,inplace=True)

In [103]:
entrenamiento_df['postulacion'] = entrenamiento_df['lo_vio'] + entrenamiento_df['postulacion']

In [104]:
entrenamiento_df.head()

,idaviso,idpostulante,lo_vio,postulacion
0,1112400775,kPjr2K0,1.0,1.0
1,1112426664,GNOwZmb,1.0,1.0
2,1112445738,lDLxajz,1.0,1.0
3,1112416123,owzZOqe,1.0,1.0
4,1112443938,NGk68l,1.0,1.0


In [105]:
entrenamiento_df = entrenamiento_df[entrenamiento_df['postulacion']==0]

In [106]:
entrenamiento_df = entrenamiento_df.drop('lo_vio',axis=1)

Juntamos los no_postulantes con los postulantes para armar el set de entrenamiento

In [107]:
entrenamiento_df = entrenamiento_df.merge(postulantes_df,how='outer',on=['idpostulante','idaviso'])

In [108]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion_x,postulacion_y
0,1112513555,NzrLwKJ,0.0,NaN
1,1111882745,N6qMwV,0.0,NaN
2,1112362954,Nzr3zvL,0.0,NaN
3,1112146217,jkj13j0,0.0,NaN
4,1112400202,ZDrmWxY,0.0,NaN


In [109]:
entrenamiento_df['postulacion_x'].fillna(0,inplace=True)
entrenamiento_df['postulacion_y'].fillna(0,inplace=True)

La idea es sumar las columnas duplicadas, ya que si habia una postulacion para esa misma dupla idaviso-idpostulante, predomina la postulacion(1) sobre la no_postulacion(0)

In [110]:
entrenamiento_df['postulacion'] = entrenamiento_df['postulacion_x'] + entrenamiento_df['postulacion_y']

In [111]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion_x,postulacion_y,postulacion
0,1112513555,NzrLwKJ,0.0,0.0,0.0
1,1111882745,N6qMwV,0.0,0.0,0.0
2,1112362954,Nzr3zvL,0.0,0.0,0.0
3,1112146217,jkj13j0,0.0,0.0,0.0
4,1112400202,ZDrmWxY,0.0,0.0,0.0


In [112]:
entrenamiento_df.drop(['postulacion_x','postulacion_y'],axis='columns',inplace=True)

In [113]:
entrenamiento_df['postulacion'].value_counts()

1.0    3277072
0.0    2998008
Name: postulacion, dtype: int64

In [114]:
entrenamiento_df['postulacion'] = pd.Categorical(entrenamiento_df['postulacion'])
entrenamiento_df['postulacion'] = entrenamiento_df['postulacion'].cat.codes
entrenamiento_df['postulacion'].value_counts()

1    3277072
0    2998008
Name: postulacion, dtype: int64

In [115]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion
0,1112513555,NzrLwKJ,0
1,1111882745,N6qMwV,0
2,1112362954,Nzr3zvL,0
3,1112146217,jkj13j0,0
4,1112400202,ZDrmWxY,0


In [116]:
entrenamiento_df.tail()

,idaviso,idpostulante,postulacion
6275075,1112274234,zvxzrWm,1
6275076,1112279048,zvxzrWm,1
6275077,1112280433,zvxzrWm,1
6275078,1112289195,zvxzrWm,1
6275079,1112290025,zvxzrWm,1


Sacamos pares idaviso-idpostulante duplicados y que aparecen en test_final

In [118]:
entrenamiento_df = entrenamiento_df.drop_duplicates(['idaviso','idpostulante'])

In [119]:
test_final_df = pd.read_csv('test_final_100k.csv')

Obtenemos los pares en comun entre nuestro set de entrenamiento y el test final

In [120]:
merge = entrenamiento_df.merge(test_final_df,on=['idpostulante','idaviso'],how='inner')

In [121]:
merge.shape

(13893, 4)

In [122]:
merge.head()

,idaviso,idpostulante,postulacion,id
0,1112406254,5bQaeZ,0,33984
1,1112424848,NkeQYJ,0,43241
2,1112094756,VNr0xb2,0,14479
3,1112476948,86zN4M,0,92838
4,1112210505,eOv8mo,0,18189


Vamos a asingar una columna con uno para luego poder identificarlos y filtrarlos

In [73]:
merge['en_test_final']=1

NameError: name 'merge' is not defined

In [124]:
merge = merge.drop(['postulacion','id'],1)

In [125]:
merge.head()

,idaviso,idpostulante,en_test_final
0,1112406254,5bQaeZ,1
1,1112424848,NkeQYJ,1
2,1112094756,VNr0xb2,1
3,1112476948,86zN4M,1
4,1112210505,eOv8mo,1


Realizando este merge, tenemos todos los datos juntos, pero ahora las duplas que queremos extraer estan identificadas con un 1.

In [126]:
entrenamiento_df = entrenamiento_df.merge(merge,on=['idpostulante','idaviso'],how='outer')

In [127]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion,en_test_final
0,1112513555,NzrLwKJ,0,NaN
1,1111882745,N6qMwV,0,NaN
2,1112362954,Nzr3zvL,0,NaN
3,1112146217,jkj13j0,0,NaN
4,1112400202,ZDrmWxY,0,NaN


In [128]:
entrenamiento_df.en_test_final.hasnans

True

In [129]:
entrenamiento_df.en_test_final.fillna(0,inplace=True)

In [130]:
entrenamiento_df.shape

(6274687, 4)

In [131]:
entrenamiento_df = entrenamiento_df[entrenamiento_df.en_test_final==0]

In [132]:
entrenamiento_df.shape

(6260794, 4)

In [133]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion,en_test_final
0,1112513555,NzrLwKJ,0,0.0
1,1111882745,N6qMwV,0,0.0
2,1112362954,Nzr3zvL,0,0.0
3,1112146217,jkj13j0,0,0.0
4,1112400202,ZDrmWxY,0,0.0


In [135]:
entrenamiento_df.drop('en_test_final',1,inplace=True)

In [134]:
entrenamiento_df.postulacion.value_counts()

1    3263194
0    2997600
Name: postulacion, dtype: int64

Para exportar e importar entrenamiento

In [44]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso
0,1112357600,vV3EwGe,1,0,1,2,0
1,1112325755,eDaqp2,1,1,0,3,2
2,1112304011,ow5zeY3,1,3,1,3,3
3,1112381569,A3X4Xxm,1,0,0,3,3
4,1112352275,VNeWK4D,1,1,0,2,1


# Para exportar e importar Set de  Entrenamiento Sin Features

entrenamiento_df.to_csv('entrenamiento_df.csv')

In [78]:
entrenamiento_df=pd.read_csv('entrenamiento_df.csv')
entrenamiento_df.drop('Unnamed: 0',axis=1,inplace=True)

# Armo la Matriz de Features

liberamos memoria de archivos que ya no vamos a utilizar

In [68]:
del postulantes_edu_df
del postulantes_gye_df
del postulantes_edad_trucha
del avisos_detalle_missing
del postulaciones_df2
del merge

Tomamos un sample de tamaño limitado para optimizar el armado de Features.

In [71]:
entrenamiento_df = entrenamiento_df[entrenamiento_df.postulacion==1].sample(50000).append(entrenamiento_df[entrenamiento_df.postulacion==0].sample(50000))

In [72]:
entrenamiento_df.postulacion.value_counts()

1    50000
0    50000
Name: postulacion, dtype: int64

Obtengo un dataframe con los avisos y sus postulaciones con informacion completa

In [ ]:
merge_detalle_postulaciones_df = postulaciones_df.merge(avisos_detalle_filtrados_df, on = 'idaviso',how='inner')

In [ ]:
merge_detalle_postulaciones_df.head(2)

In [ ]:
postulantes = postulantes_filtrados_efectivos_df.groupby('idpostulante')

In [ ]:
merge_detalle_postulaciones_df = merge_detalle_postulaciones_df.merge(avisos_detalle_filtrados_df.drop(columns=['nombre_zona','tipo_de_trabajo','nivel_laboral','denominacion_empresa','nombre_area','titulo'],axis=1),how='inner',on='idaviso')

# Para exportar e importar merge_detalle_postulaciones_df

In [85]:
merge_detalle_postulaciones_df.to_csv('merge_detalle_postulaciones_df.csv')

In [86]:
merge_detalle_postulaciones_df = pd.read_csv('merge_detalle_postulaciones_df.csv')

In [87]:
merge_detalle_postulaciones_df.drop('Unnamed: 0',1,inplace=True)

In [88]:
merge_detalle_postulaciones_df.head()

,idaviso,idpostulante,fechapostulacion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,titulo
0,1112248724,NjlD,2018-01-19 07:39:16,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,Analista de CRM y Marketing Digital
1,1112248724,MV6eM8N,2018-01-30 23:44:30,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,Analista de CRM y Marketing Digital
2,1112248724,5LzQkN,2018-01-15 07:23:49,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,Analista de CRM y Marketing Digital
3,1112248724,8M2rOBz,2018-01-30 22:20:45,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,Analista de CRM y Marketing Digital
4,1112248724,Dr6V3Da,2018-01-23 15:11:35,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Marketing,Lo Jack,Analista de CRM y Marketing Digital


# Comenzamos a armar los features

En primer lugar agrupo todos los avisos a los que se postulo un usuario

In [89]:
grupo_postulantes = merge_detalle_postulaciones_df.groupby('idpostulante')

En segundo lugar hago que 'idaviso' sea el indice para poder acceder de forma directa.

In [90]:
grupo_avisos = avisos_detalle_filtrados_df.groupby('idaviso')

Armo lista de los usuarios que se postularon a algun aviso del que yo tenga datos en avisos_detalle_filtrados_df

In [91]:
usuarios_con_postulaciones = grupo_postulantes.groups.keys()

Formato necesario para acceder de forma directa a los idpostulantes.

In [92]:
postulantes = postulantes_filtrados_efectivos_df.groupby('idpostulante')

### Funcion para reducir el ruido en los features de area, empresa e interes titulo

In [93]:
def categorizarFeatures(num):
    if num>0 and num<=3:
        return 1
    if num>3 and num<=10:
        return 2
    if num>10:
        return 3
    return num

# Feature Postulacion A Area

## A ese Area En esa Zona

In [94]:
def armarFeaturePostulacionesUsuarioAEseAreaEnEsaZona(idpostulante,idaviso,ajuste):
    if (idpostulante not in usuarios_con_postulaciones):
        return 0
    area=grupo_avisos.get_group(idaviso)['nombre_area'].iloc[0]
    zona=grupo_avisos.get_group(idaviso)['nombre_zona'].iloc[0]
    postulaciones_de_usuario = grupo_postulantes.get_group(idpostulante).groupby('nombre_area')
    if area in postulaciones_de_usuario['nombre_area'].groups.keys():
        zonas=postulaciones_de_usuario.get_group(area)['nombre_zona'].value_counts()
        if zonas.index.contains(zona):
            return categorizarFeatures((zonas[zona]-ajuste))
    return 0

In [95]:
feature = entrenamiento_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEseAreaEnEsaZona(row['idpostulante'], row['idaviso'],row['postulacion']),axis=1)

KeyboardInterrupt: 

In [ ]:
entrenamiento_df['postulaciones_a_ese_area_en_esa_zona'] = pd.DataFrame(feature)

# Feature Postulacion A Empresa

In [ ]:
def armarFeaturePostulacionesUsuarioAEsaEmpresa(idpostulante,idaviso,ajuste):
    if (idpostulante in usuarios_con_postulaciones) == False:
        return 0
    empresa = grupo_avisos.get_group(idaviso)['denominacion_empresa'].iloc[0]
    empresas = grupo_postulantes.get_group(idpostulante)['denominacion_empresa'].value_counts()
    if empresas.index.contains(empresa):
        return categorizarFeatures((empresas[empresa]-ajuste))
    return 0

In [ ]:
feature = entrenamiento_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEsaEmpresa(row['idpostulante'], row['idaviso'],row['postulacion']),axis=1)

In [ ]:
entrenamiento_df['postulaciones_a_esa_empresa'] = pd.DataFrame(feature)

# Feature Edad- Genero -Area

In [ ]:
merge_detalle_postulaciones_y_usuarios=merge_detalle_postulaciones_df.merge(postulantes_filtrados_df,on='idpostulante')

In [ ]:
merge_detalle_postulaciones_y_usuarios.drop(['nivel_laboral','tipo_de_trabajo','nombre_zona','denominacion_empresa','idpostulante','idaviso','titulo'],axis=1,inplace=True)

In [ ]:
postulantes_filtrados_df=postulantes_filtrados_df.set_index('idpostulante')

In [ ]:
grupo_por_edad_sexo = merge_detalle_postulaciones_y_usuarios.groupby(['edad','sexo'])

In [ ]:
grupo_por_edad_sexo = grupo_por_edad_sexo.nombre_area.value_counts()

In [ ]:
del merge_detalle_postulaciones_y_usuarios

In [ ]:
cant_de_avisos_por_area = avisos_detalle_filtrados_df.nombre_area.value_counts()

In [ ]:
grupo_por_edad_sexo.head()

In [ ]:
cant_de_avisos_por_area.head()

In [ ]:
def obtener_valoracion_feature(area,sexo,edad):
    try:
        result = ((grupo_por_edad_sexo[edad][sexo][area]/cant_de_avisos_por_area[area])/((grupo_por_edad_sexo[edad][sexo]/cant_de_avisos_por_area).sum()))
    except (KeyError, IndexError):
        return 0
    return abs(result*1000)

In [ ]:
def categorizarFeaturesEspeciales(num):
    if num<=0:
        return 0
    if num>0 and num<3:
        return 1
    if num>=3 and num<=7:
        return 2
    if num>7 and num<=20:
        return 3
    if num>20 and num <=35:
        return 4
    if num>35 and num <=50:
        return 5
    return 6

In [ ]:
""" Funcion que obtiene en base a la edad, area y sexo, un puntaje especifico."""
def FeatureEdadGeneroArea(idpostulante,idaviso):
    area_aviso = grupo_avisos.get_group(idaviso)['nombre_area'].iloc[0]
    genero = postulantes_filtrados_df.loc[idpostulante]['sexo']
    edad = postulantes_filtrados_df.loc[idpostulante]['edad']
    resultado_parcial =  obtener_valoracion_feature(area_aviso,genero,edad)
    resultado = categorizarFeaturesEspeciales(resultado_parcial)
    return resultado



In [ ]:
feature = entrenamiento_df.apply(lambda row: FeatureEdadGeneroArea(row['idpostulante'], row['idaviso']),axis=1)

In [ ]:
entrenamiento_df['relacion_edad_genero_aviso'] = pd.DataFrame(feature.fillna(0))


# Feature Titulo

In [ ]:
import unicodedata
def shave_marks(txt):
    """This method removes all diacritic marks from the given string"""
    norm_txt = unicodedata.normalize('NFD', txt)
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c))
    return unicodedata.normalize('NFC', shaved)

In [ ]:
from collections import Counter
def armarFeatureInteresEnPalabrasDeTitulo(idpostulante, idaviso, ajuste):
    if (idpostulante in usuarios_con_postulaciones) == False:
        return 0
    palabras_en_titulos_de_sus_postulaciones = shave_marks(" ".join(grupo_postulantes.get_group(idpostulante)['titulo'].values.tolist()).lower())
    #Saco comas, parentesis, sufijos etc. Para abarcar mas palabras
    titulo_aviso = shave_marks(grupo_avisos.get_group(idaviso)['titulo'].iloc[0].lower()).replace("(",' ').replace(")",' ').replace(",",' ').replace('/',' ').replace(' para ',' ').replace('-',' ').replace(' con ',' ').replace('experiencia',' ').replace('zona',' ').replace('villa',' ').replace('barrio',' ').replace('ero ',' ').replace('era ',' ').replace('cion ',' ').replace('ativo ',' ').replace('ativa ',' ').replace('dor ',' ').replace('dora ',' ').replace('dores ',' ').replace('doras ',' ').replace('ista ',' ').replace('or ',' ').replace('ora ',' ').replace('ico ',' ').replace('ica ',' ').replace('icos ',' ').replace('icas ',' ').replace('time',' ').replace('area',' ').split(" ")
    contador=0
    for x in titulo_aviso:
        if  len(x)>2 and x!='' and palabras_en_titulos_de_sus_postulaciones.count(x) > 0:
            contador = contador + palabras_en_titulos_de_sus_postulaciones.count(x) - ajuste
    return categorizarFeatures(contador)

In [ ]:
feature = entrenamiento_df.apply(lambda row: armarFeatureInteresEnPalabrasDeTitulo(row['idpostulante'], row['idaviso'],row['postulacion']),axis=1)

In [ ]:
entrenamiento_df['interes_en_titulo_de_aviso'] = pd.DataFrame(feature)

### Feature Nivel Laboral y Tipo de Trabajo

In [219]:
entrenamiento_df = entrenamiento_df.merge(avisos_detalle_filtrados_df.drop(['titulo','nombre_zona','nombre_area','denominacion_empresa'],1),how='inner',on='idaviso')

# Para exportar e importar Set de Entrenamiento con Features

In [221]:
entrenamiento_df.to_csv('entrenamiento_con_features_100k.csv')

In [225]:
entrenamiento_df = pd.read_csv('entrenamiento_con_features_100k.csv')
entrenamiento_df.drop('Unnamed: 0',1,inplace=True)

In [226]:
entrenamiento_df.head()

,idaviso,idpostulante,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso,tipo_de_trabajo,nivel_laboral
0,1112357600,vV3EwGe,1,0,1,2,0,1,4
1,1112357600,ZD2NNXb,1,2,1,2,2,1,4
2,1112357600,ew4dPr,1,2,1,2,1,1,4
3,1112357600,dY2k14D,1,1,1,3,0,1,4
4,1112357600,dY9JjQD,1,0,1,2,2,1,4


# Comenzamos a dar formato para realizar el Fit del algoritmo. Creamos X e y

In [188]:
X = entrenamiento_df

In [189]:
X.head()

,idaviso,idpostulante,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso,tipo_de_trabajo,nivel_laboral
0,1112357600,vV3EwGe,1,0,1,2,0,1,4
1,1112357600,ZD2NNXb,1,2,1,2,2,1,4
2,1112357600,ew4dPr,1,2,1,2,1,1,4
3,1112357600,dY2k14D,1,1,1,3,0,1,4
4,1112357600,dY9JjQD,1,0,1,2,2,1,4


Me quedo con el indice como "id" de la tupla postulante-aviso.

In [190]:
X.drop(columns=['idpostulante','idaviso'],axis=1,inplace=True)

In [191]:
X.head()

,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso,tipo_de_trabajo,nivel_laboral
0,1,0,1,2,0,1,4
1,1,2,1,2,2,1,4
2,1,2,1,2,1,1,4
3,1,1,1,3,0,1,4
4,1,0,1,2,2,1,4


In [192]:
X['postulacion'].value_counts()

1    50000
0    50000
Name: postulacion, dtype: int64

In [193]:
y = X['postulacion']
X.head()

,postulacion,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso,tipo_de_trabajo,nivel_laboral
0,1,0,1,2,0,1,4
1,1,2,1,2,2,1,4
2,1,2,1,2,1,1,4
3,1,1,1,3,0,1,4
4,1,0,1,2,2,1,4


In [194]:
X = X.drop('postulacion',axis=1)

In [195]:
X.shape

(100000, 6)

In [196]:
y.shape

(100000,)

# Armamos la matriz de prediccion a partir del archivo de test

In [173]:
test_final_df = pd.read_csv('test_final_100k.csv')

In [174]:
test_final_df.head(2)

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL


### Feature area en zona

In [258]:
feature = test_final_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEseAreaEnEsaZona(row['idpostulante'], row['idaviso'],0),axis=1)
test_final_df['postulaciones_a_ese_area_en_esa_zona']=pd.DataFrame(feature)

In [259]:
test_final_df.head(2)

,id,idaviso,idpostulante,postulaciones_a_ese_area_en_esa_zona
0,0,739260,6M9ZQR,0
1,1,739260,6v1xdL,2


### Feature postulaciones a esa empresa

In [260]:
feature = test_final_df.apply(lambda row: armarFeaturePostulacionesUsuarioAEsaEmpresa(row['idpostulante'], row['idaviso'],0),axis=1)

In [261]:
test_final_df['postulaciones_a_esa_empresa']=pd.DataFrame(feature)

In [262]:
test_final_df.head(2)

,id,idaviso,idpostulante,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa
0,0,739260,6M9ZQR,0,0
1,1,739260,6v1xdL,2,0


### Feature Edad-Genero-Area

In [266]:
feature = test_final_df.apply(lambda row: FeatureEdadGeneroArea(row['idpostulante'], row['idaviso']),axis=1)

In [267]:
test_final_df['relacion_edad_genero_aviso'] = pd.DataFrame(feature.fillna(0))

### Feature Interes en Titulo

In [268]:
feature = test_final_df.apply(lambda row: armarFeatureInteresEnPalabrasDeTitulo(row['idpostulante'], row['idaviso'],0),axis=1)

In [269]:
test_final_df['interes_en_titulo_de_aviso']=pd.DataFrame(feature)

In [270]:
test_final_df.head(2)

,id,idaviso,idpostulante,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso
0,0,739260,6M9ZQR,0,0,3,0
1,1,739260,6v1xdL,2,0,2,0


### Feature Nivel Laboral  y Tipo de Trabajo

In [ ]:
test_final_df = test_final_df.merge(avisos_detalle_filtrados_df.drop(['titulo','nombre_zona','nombre_area','denominacion_empresa'],1),how='inner',on='idaviso')

### Formato final

In [271]:
test_final_df = test_final_df.drop(['id','idaviso','idpostulante'],axis=1)

In [272]:
test_final_df.head()

,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso
0,0,0,3,0
1,2,0,2,0
2,0,0,3,0
3,0,0,1,0
4,0,0,1,0


### Para Exportar e Importar Test Final con Features

In [216]:
test_final_df.to_csv('test_final_con_features_df.csv')

In [213]:
test_final_df = pd.read_csv('test_final_con_features_df.csv')
test_final_df.drop('Unnamed: 0',1,inplace=True)

In [214]:
avisos_detalle_filtrados_df.head()

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,Responsable de Marketing-Producto Div. Constru...,Gran Buenos Aires,1,4,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,Asesor de Servicios - Postventa Chevrolet,Gran Buenos Aires,1,4,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,Analistas de Impuestos Sr Ssr y Jr,Gran Buenos Aires,1,4,Impuestos,Lighthouse Smart
3,1112146010,Software Engineer,Gran Buenos Aires,1,4,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,Business Developer Manager,Gran Buenos Aires,1,4,Comercial,RANDSTAD


In [201]:
test_final_df.drop(['idpostulante','idaviso','id'],1,inplace=True)

In [217]:
test_final_df.head()

,id,idaviso,idpostulante,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso,tipo_de_trabajo,nivel_laboral
0,0,739260,6M9ZQR,0,0,3,0,1,1
1,1,739260,6v1xdL,2,0,2,0,1,1
2,2,739260,ezRKm9,0,0,3,0,1,1
3,3,758580,1Q35ej,0,0,1,0,1,3
4,4,758580,EAN4J6,0,0,1,0,1,3


Verificamos que coincidan las columnas

In [112]:
X.head()

,postulaciones_a_ese_area_en_esa_zona,postulaciones_a_esa_empresa,relacion_edad_genero_aviso,interes_en_titulo_de_aviso
0,0,1,2,0
1,1,0,3,2
2,3,1,3,3
3,0,0,3,3
4,1,0,2,1


# Algoritmos

In [325]:
from sklearn.model_selection import cross_val_score

# XGBoost

In [326]:
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [128]:
xgbclassiffier = xgb.XGBClassifier(max_depth=4, learning_rate=0.1)

In [39]:
scores = cross_val_score(xgbclassiffier,X,y)
scores.mean()  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8995355009472982

In [129]:
xgbclassiffier.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [132]:
pred = xgbclassiffier.predict_proba(test_final_df)

In [133]:
pred = pred[:,1]

In [157]:
list(zip(X[X.columns], xgbclassiffier.feature_importances_))

[('postulaciones_a_esa_zona', 0.20666666),
 ('postulaciones_a_ese_area', 0.15133333),
 ('postulaciones_a_esa_empresa', 0.12733333),
 ('relacion_educacion_con_aviso', 0.154),
 ('relacion_edad_con_aviso', 0.104666665),
 ('interes_en_titulo_de_aviso', 0.256)]

# Random Forest

In [332]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(criterion='entropy')

In [ ]:
scores = cross_val_score(clf,X,y)
scores.mean()  

In [333]:
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [334]:
list(zip(X[X.columns], clf.feature_importances_))

[('postulaciones_a_ese_area_en_esa_zona', 0.3423077358479879),
 ('postulaciones_a_esa_empresa', 0.05697225981167721),
 ('relacion_edad_genero_aviso', 0.06916100238234987),
 ('interes_en_titulo_de_aviso', 0.5315590019579849)]

In [335]:
pred = clf.predict_proba(test_final_df)

pred = pred[:,1]

# Perceptron

In [113]:
from sklearn.linear_model import Perceptron
ptron = Perceptron()

In [ ]:
scores = cross_val_score(ptron,X,y)
scores.mean()  

In [124]:
ptron.fit(X,y)

/home/luciano/.local/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [125]:
ptron.score(X,y)

0.89038

In [383]:
pred = ptron.predict(test_final_df)

# Bagging con Perceptron

In [114]:
from sklearn.ensemble import BaggingClassifier
bc = BaggingClassifier(base_estimator=ptron,n_estimators=50)

In [ ]:
scores = cross_val_score(bc,X,y)
scores.mean()  

In [125]:
bc.fit(X,y)

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Andres\Anaconda3\lib\site-packages\sklearn\li

BaggingClassifier(base_estimator=Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=50, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [126]:
bc.score(X,y)

0.85465

In [127]:
pred = bc.predict_proba(test_final_df)

In [128]:
pred = pred[:,1]

# Perceptron multinivel

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp = MLPClassifier()

In [ ]:
scores = cross_val_score(mlp,X,y)
scores.mean()  

In [ ]:
mlp.fit(X,y) 

In [ ]:
pred = mlp.predict_proba(test_final_df)

# KNN

In [327]:
from sklearn.neighbors import KNeighborsClassifier

In [328]:
knn = KNeighborsClassifier(n_neighbors=50)

In [ ]:
scores = cross_val_score(knn,X,y)
scores.mean()  

In [329]:
knn.fit(X,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=50, p=2,
           weights='uniform')

In [330]:
knn.score(X,y)

0.523

In [331]:
knn.predict(test_final_df)

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

# Bagging KNN

In [ ]:
from sklearn.ensemble import BaggingClassifier
bcknn = BaggingClassifier(base_estimator=knn,n_estimators=20)

In [ ]:
scores = cross_val_score(bcknn,X,y)
scores.mean()  

In [ ]:
bcknn.fit(X,y)

In [ ]:
bcknn.score(X,y)

In [ ]:
pred=bcknn.predict_proba(test_final_df)

# Naive_Bayes

In [364]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
scores = cross_val_score(nb,X,y)
scores.mean()  

In [371]:
nb.fit(X, y)

GaussianNB(priors=None)

In [372]:
nb.score(X,y)

0.82867

In [375]:
pred = nb.predict_proba(test_final_df)

# Boosting

In [197]:
from sklearn.ensemble import AdaBoostClassifier

adaboost = AdaBoostClassifier(n_estimators=100)
 

In [198]:
scores = cross_val_score(adaboost,X,y)
scores.mean()  

0.8534897512778175

In [199]:
adaboost.fit(X,y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)

In [202]:
pred = adaboost.predict_proba(test_final_df)

In [203]:
pred = pred[:,1]

In [204]:
list(zip(X, adaboost.feature_importances_))

[('postulaciones_a_ese_area_en_esa_zona', 0.11),
 ('postulaciones_a_esa_empresa', 0.05),
 ('relacion_edad_genero_aviso', 0.16),
 ('interes_en_titulo_de_aviso', 0.12),
 ('tipo_de_trabajo', 0.46),
 ('nivel_laboral', 0.1)]

# Gradient Boosting

In [346]:
from sklearn.ensemble import GradientBoostingClassifier

In [347]:
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=2, random_state=0)

In [ ]:
scores = cross_val_score(gb,X,y)
scores.mean()  

In [348]:
gb.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=1.0, loss='deviance', max_depth=2,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [349]:
pred = gb.predict_proba(test_final_df)

In [350]:
pred = pred[:,1]

# ARMADO DE SUBMIT

In [205]:
test_final_df = pd.read_csv('test_final_100k.csv')

In [206]:
test_final_df['sepostulo'] = pd.DataFrame(pred)

In [207]:
submit = test_final_df.drop(['idaviso','idpostulante'],axis=1)

In [208]:
submit.set_index('id',inplace=True)

In [209]:
submit.head()

,sepostulo
id,
0,0.494491
1,0.497447
2,0.494491
3,0.492326
4,0.492326


In [210]:
submit.sepostulo.value_counts()

0.495003    10990
0.494197     9939
0.491955     5868
0.509433     2983
0.495418     2772
0.494612     2255
0.508187     1600
0.510057     1500
0.495374     1291
0.498672     1147
0.499479     1096
0.494567     1095
0.509848     1061
0.508626     1039
0.492370     1011
0.509615      955
0.508714      952
0.508629      925
0.495509      901
0.493684      881
0.507381      841
0.508809      812
0.494491      812
0.495924      721
0.509251      681
0.497585      678
0.496431      649
0.494703      611
0.492326      594
0.495118      563
            ...  
0.505667        1
0.503346        1
0.514646        1
0.505670        1
0.504470        1
0.500890        1
0.510826        1
0.507842        1
0.505831        1
0.503515        1
0.508029        1
0.506497        1
0.507878        1
0.510962        1
0.510729        1
0.509989        1
0.507781        1
0.503205        1
0.504611        1
0.505647        1
0.506946        1
0.507276        1
0.507295        1
0.503663        1
0.508531  

In [211]:
submit.head()

,sepostulo
id,
0,0.494491
1,0.497447
2,0.494491
3,0.492326
4,0.492326


In [212]:
submit.to_csv('Submit_file.csv')